<a href="https://colab.research.google.com/github/ngolla/video-captioning/blob/master/Copy_of_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
from google.colab import drive
from pathlib import Path
import numpy as np

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_path=Path('/content/drive/My Drive')
list(drive_path.glob('*'))

[PosixPath('/content/drive/My Drive/Colab Notebooks'),
 PosixPath('/content/drive/My Drive/YouTubeClips.tar'),
 PosixPath('/content/drive/My Drive/video_corpus.csv'),
 PosixPath('/content/drive/My Drive/.ipynb_checkpoints'),
 PosixPath('/content/drive/My Drive/VideoArrays'),
 PosixPath('/content/drive/My Drive/model1.h5'),
 PosixPath('/content/drive/My Drive/splitdata'),
 PosixPath('/content/drive/My Drive/splitset'),
 PosixPath('/content/drive/My Drive/test.dat'),
 PosixPath('/content/drive/My Drive/tokens_word_index.txt'),
 PosixPath('/content/drive/My Drive/tokens_index_word.txt')]

In [ ]:
split_path=drive_path.joinpath('splitset')
list(split_path.glob('*'))

[PosixPath('/content/drive/My Drive/splitset/train.dat'),
 PosixPath('/content/drive/My Drive/splitset/test.dat'),
 PosixPath('/content/drive/My Drive/splitset/tokens_word_index.txt'),
 PosixPath('/content/drive/My Drive/splitset/tokens_index_word.txt')]

In [ ]:
train_file=split_path.joinpath('train.dat')
val_file=split_path.joinpath('test.dat')

In [ ]:
(vid_val, cap_val)=pickle.load(open( val_file, "rb" ))
(vid_train, cap_train)=pickle.load(open( train_file, "rb" ))

In [ ]:
print(len(vid_train), vid_train[0].shape)
print(len(vid_val), vid_val[0].shape)

8000 (80, 4096)
2000 (80, 4096)


In [ ]:
vid_train=np.array(vid_train)
cap_train=np.array(cap_train)

In [ ]:
vid_val=np.array(vid_val)
#cap_val=np.array(cap_val)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# returns train, inference_encoder and inference_decoder models
def basic_enc_dec(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    # return all models
    return model, encoder_model, decoder_model

In [ ]:
#split_path.joinpath('tokens_word_index.txt')
token=pickle.load(open(split_path.joinpath('tokens_word_index.txt'),"rb"))
len(token.items())

3434

In [ ]:
cap_train[0].shape[0], vid_train.shape[2]

(35, 4096)

In [ ]:
vocab_size=len(token.items())+1
#dim_embedding=64
maxlen=cap_train[0].shape[0]

In [ ]:
model, enc, dec = basic_enc_dec(vid_train.shape[2], vocab_size, maxlen)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 4096)] 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 3435)] 0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 35), (None,  578480      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 35), ( 485940      input_2[0][0]                    
                                                                 lstm[0][1]            

In [ ]:
x2 = np.hstack([np.zeros((len(cap_train), 1)), cap_train])
x2 = x2[:, :-1]

In [ ]:
#Convert to (no of captions)x(no of words)x(vocab size)
from keras.utils.np_utils import to_categorical   

#x2_in = to_categorical(x2, num_classes = vocab_size)
x2 = to_categorical(x2, num_classes = vocab_size)
outputs = to_categorical(cap_train, num_classes = vocab_size)
#print(x2_in.shape, outputs.shape)
print(x2.shape, outputs.shape)

In [ ]:
del cap_train

In [ ]:
x2_val = np.hstack([np.zeros((cap_val.shape[0], 1)), cap_val])
x2_val = x2_val[:, :-1]

x2_val = to_categorical(x2_val, num_classes = vocab_size)
outputs_val = to_categorical(cap_val, num_classes = vocab_size)

print(x2_val.shape, outputs_val.shape)

In [ ]:
del cap_val

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
model.compile(optimizer=RMSprop(lr=7e-4), loss='categorical_crossentropy')
#history=model.fit([vid, x2_in], outputs, epochs = 1)
callback = EarlyStopping(monitor='loss', patience=3)
history=model.fit([vid_train, x2], outputs, validation_data=([vid_val, x2_val], outputs_val), epochs = 100,callbacks=[callback], verbose=0)
#history=model.fit([vid_train, x2_in],outputs,batch_size=100,epochs=1, validation_split=0.2)

 This callback will stop the training when there is no improvement in  the validation loss for three consecutive epochs. 

In [ ]:
history.history
model.save(split_path.joinpath('model_100.h5'))
enc, dec
enc.save(split_path.joinpath('encoder.h5'))
dec.save(split_path.joinpath('decoder.h5'))